# Let's Begin 
Load the magic commands to autoreload files

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import necessary packages

In [ ]:
from pathlib import Path
from fastai.vision import *
from fastai.metrics import error_rate

Define the different paths required for this project

In [ ]:
path = Path('../input/waste-classification-data/dataset/DATASET'); 
path

In [ ]:
pathTest = path/'TEST'
pathTrain = path/'TRAIN'
pathTest.ls()

Check the number of files available in the path

In [ ]:
numberOfFiles = (pathTest/'O').ls()
numberOfFiles[:5]

Create the imagedatabunch

In [ ]:
data = ImageDataBunch.from_folder(path, ds_tfms=get_transforms(), valid_pct=0.2, size=224).normalize(imagenet_stats)

View the data in your databunch

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

Print the number of categories available for your dataset

In [ ]:
print(data.classes)
len(data.classes),data.c

Create the neural network

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

Train the network

In [ ]:
learn.fit_one_cycle(4)

Save the trained network

In [ ]:
learn.model_dir = "/kernel/output"
learn.save('stage-1', return_path=True)

Look at the classification result

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

Check the top losses

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

Check the confusion metric

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

Check the labels on which the network was most confused with

In [ ]:
interp.most_confused(min_val=2)

Unfreeze the network

In [ ]:
learn.unfreeze()

Fit one cycle

In [ ]:
learn.fit_one_cycle(1)

Load the previously saved model

In [ ]:
learn.load('stage-1');

Plot the learning rate

In [ ]:
learn.lr_find()
learn.recorder.plot()

Train some more

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

Export the final model

In [ ]:
learn.path = Path("/kernel")
learn.export()

## Testing time

Create an image list and select one image

In [ ]:
testImageList = ImageList.from_folder(pathTest/'O')
image = open_image(testImageList.items[0])
image

Load the model and get the predicted class, predicted label and probability of prediction

In [ ]:
model = load_learner(learn.path)
className, label, probability = model.predict(image)

In [ ]:
className, label, probability